In [ ]:
from environment import BuyKnowledge, SellKnowledge
from preprocessing import donchian_signals, wrapper_func
from stable_baselines3 import PPO
import pandas as pd

In [ ]:
import exchange_api as ex

# Check current balance and remove currency balances

In [ ]:
bal = ex.get_balance()
usd = float(bal.pop(‘USD’, None))

# Generate Donchian buy signals and calculate 20 day return

In [ ]:
sp500 = pd.read_excel('SP500.xlsx')
sp500 = sp500.astype(str)

In [ ]:
buy_symbols = []
sideways_20 = {}
current_price = {} 
for symbol in sp500['Symbol']: 
    df = donchian_signals(symbol)
    sideways_20[symbol] = df.iloc[-1].close/ df.iloc[0].close -1
    current_price[symbol] = df.iloc[-1].close
    if df.iloc[-1].buy_signal and not df.iloc[-2].buy_signal:
        buy_symbols.append(symbolusd)

# Sell logic

## Get trade history
Some minor changes to lines involving the "trades" dataframe may be required depending on the format of the trade history data provided by your exchange  

In [ ]:
trades = ex.get_past_trades()
for col in ['price','amount','fee_amount']:
    trades[col] = pd.to_numeric(trades[col])

## Calculate sell return (profit of selling a stock today), execute stop-loss module (refer to source paper for more info), and get data for the remaining stocks

In [ ]:
sell_symbols = []
sell_returns = []
sell_data = pd.DataFrame()
for symbol in bal: 
    # Calculate average buy price since the last sell point
    last_sell_index = trades[(trades['symbol']==symbol+'USD') & (trades['type'] == 'Sell')].index[0]
    last_buys = trades[trades['symbol']==symbol+'USD'].loc[:last_sell_index-1]
    last_buys['spent'] = last_buys['price'] * last_buys['amount'] + last_buys['fee_amount']
    last_buy_price = last_buys['spent'].sum()/last_buys['amount'].sum()
    
    symbol = symbol.lower()
    sell_return = current_price[symbol] / last_buy_price -1
    # Stop Loss: sideways and dips
    if sideways_20[symbol] <0.1 or sell_return < 0.1:
        try:
            amount = float(bal[symbol.upper()])
            print(ex.stop_loss_sell(symbol, amount))
        except:
            print(f'Sell {symbol} failed')

    # Get data for other held symbols
    else:
        _, df = wrapper_func(symbol, days = 390, calculate_returns = False)
        sell_symbols.append(symbol)
        sell_returns.append(sell_return)
        sell_data = pd.concat([sell_data, df.iloc[-1,:-2].reset_index(drop=True)], axis = 1)

sell_data = sell_data.T
sell_data['sell_return'] = sell_returns

## Evaluate stocks not sold by stop-loss

In [ ]:
if len(sell_symbols) > 0:
    # Initialise deployment environment and model
    env = SellKnowledge(sell_data, train_mode = False)
    model = PPO.load('sell_knowledge_agent')
    obs,info = env.reset()
    done = False
    actions = []

    # Run model to get predicted probabilities 
    while True:
        try:
            action, _states = model.predict(pd.to_numeric(obs))
            actions.append(action)
        except:
            print(f'Prediction failed at {env.cur_step}')
            actions.append([-1,-1])
        if done:
            break
        obs, reward, done, trunc, info = env.step(action)

    # Sell if P(sell) - P(hold) > 0.85
    rank_sells = {}
    for i in range(len(sell_symbols)):
        rank_sells[sell_symbols[i]] = actions[i]
    rank_sells = pd.DataFrame(rank_sells).T.reset_index()
    rank_sells['diff'] = rank_sells[0] - rank_sells[1]
    rank_sells.to_csv('rank_sells.csv', index = False)
    rank_sells = rank_sells[rank_sells['diff'] > 0.85]

    # Sell the corresponding symbols 
    for i in range(len(rank_sells)):
        symbolusd = rank_sells.iloc[i]['index']
        symbol = symbolusd[:-3]
        try:
            amount = float(bal[symbol.upper()])
            print(ex.new_market_sell(symbol, amount))
        except:
            print(f'Sell {symbol} failed')

# Buy logic

## Get data for stocks that triggered buy signals

In [ ]:
for symbol in buy_symbols:
    _, df = wrapper_func(symbol[:-3], days = 390, calculate_returns = False)
    buy_data = pd.concat([buy_data, df.iloc[-1,:-2].reset_index(drop=True)], axis = 1)
    
buy_data = buy_data.T

## Evaluate stocks to buy

In [ ]:
buy_amount = 100 # x units of currency, adjust as desired

In [ ]:
if len(buy_symbols) > 0:
    # Initialise deployment environment and model
    env = BuyKnowledge(buy_data, train_mode = False)
    model = PPO.load('buy_knowledge_agent')
    obs,info = env.reset()
    done = False
    actions = []

    # Run model to get predicted probabilities 
    while True:
        try:
            action, _states = model.predict(pd.to_numeric(obs))
            actions.append(action)
        except:
            print(f'Prediction failed at {env.cur_step}')
            actions.append([-1,-1])
        if done:
            break
        obs, reward, done, trunc, info = env.step(action)

    # Prioritize buy signals with higher probability of high returns
    rank_buys = {}
    for i in range(len(buy_symbols)):
        rank_buys[buy_symbols[i]] = actions[i]
    rank_buys = pd.DataFrame(rank_buys).T.reset_index()
    rank_buys['diff'] = rank_buys[0] - rank_buys[1]
    rank_buys = rank_buys.sort_values(0, ascending = False).reset_index(drop = True) 
    rank_buys.to_csv('rank_buys.csv', index = False)
    rank_buys = rank_buys[rank_buys['diff'] > 0] # increase threshold for lower risk

    # Buy the corresponding symbols 
    for i in range(len(rank_buys)):
        row = rank_buys.iloc[i]
        symbol = row['index']
        try:
            print(ex.new_market_buy(symbol, buy_amount))
        except:
            print('Buy failed, remaining buy signals saved to csv')
            rank_buys.iloc[i:].to-csv('remaining_buys.csv', index = False)
            break 